In [ ]:
# version 2 (final version) - 修改 gradio介面, 累積提問與回答

# pip3 install langchain_ollama gradio pypdf chromadb
# ollama pull nomic-embed-text

# pip3 pip install ipywidgets
# jupyter nbextension enable --py widgetsnbextension
# pip3 install -U langchain-ollama

In [23]:
import os, sys
sys.path.append('.')

from langchain_ollama import OllamaLLM
import gradio as gr

# 初始化一個全局的問答歷史列表
chat_history = []

# Ollama 聊天模型函數
def ollama_llm(question):
    formatted_prompt = f"總是用繁體中文回答！\n\nQuestion: {question}"
    llm = OllamaLLM(model="llama3.1:70b", base_url="http://localhost:11434")
    
    try:
        response = llm.generate(prompts=[formatted_prompt])
        return response.generations[0][0].text
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio 的主邏輯函數，用於處理問題並更新聊天歷史
def get_important_facts(question):
    # 獲取模型回答
    answer = ollama_llm(question)
    
    # 更新聊天歷史
    chat_history.append((question, answer))
    
    # 將所有聊天歷史格式化成單一字符串，顯示於輸出
    chat_output = ""
    for q, a in chat_history:
        chat_output += f"**問題**: {q}\n\n**回答**: {a}\n\n"
    
    return chat_output

# 創建 Gradio 應用介面
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="請輸入您的問題"),
  outputs="markdown",  # 使用 markdown 格式顯示聊天歷史
  title="Ollama Chat",
  description="使用 Llama3 模型進行問答，顯示完整對話歷史",
)

# 啟動 Gradio 應用
iface.launch()
